[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/semantic-search/sparse/bm25/bm25-vector-generation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/semantic-search/sparse/bm25/bm25-vector-generation.ipynb)

# Generating hybrid Sparse-Dense embeddings using BM25 and sentence transformers

[![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/fast-link.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/pinecone/sparse/bm25/bm25-quora.ipynb)

## Overview

[BM25](https://en.wikipedia.org/wiki/Okapi_BM25) is a popular technique for retrieving text. It uses term frequencies to determine the relative importance of the term to the query. It is simple but effective and only requires knowing the number of documents in a corpus and the frequency of terms across documents. In the following guide, we will show how to use BM25 with Pinecone's sparse-dense vectors for use in hybrid search.

Skip the embedding creation step by using the [companion guide](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/bm25/bm25-quora.ipynb).

## Prerequisites

We'll install the required libraries:

In [1]:
!pip install -qU \
    pinecone-text \
    sentence-transformers \
    torch \
    pandas==2.0.2 \
    datasets==2.12.0 \
    pyarrow

In [2]:
from tqdm.auto import tqdm

## Quora Dataset

First We'll load the popular [Quora dataset](https://huggingface.co/datasets/quora).
The Quora dataset is composed of question pairs, and the task is to determine if the questions are duplications of each other (have the same meaning).

In [3]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("quora")

Found cached dataset quora (/Users/amnoncatav/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)


  0%|          | 0/1 [00:00<?, ?it/s]

Let's look on a single record in this dataset

In [4]:
raw_df = dataset["train"].to_pandas()

raw_df.iloc[7].to_dict()

{'questions': {'id': array([15, 16], dtype=int32),
  'text': array(['How can I be a good geologist?',
         'What should I do to be a great geologist?'], dtype=object)},
 'is_duplicate': True}

As you can see above, each record contains two questions and a binary label that indicates whether the questions duplicates of each other.

Before we start to create embedding from our data, we need to process it. First, we need to convert the JSON column into distinct columns within our dataframe.

In [5]:
raw_df["q1"] = raw_df["questions"].apply(lambda q: q["text"][0])
raw_df["q2"] = raw_df["questions"].apply(lambda q: q["text"][1])
raw_df["id1"] = raw_df["questions"].apply(lambda q: q["id"][0])
raw_df["id2"] = raw_df["questions"].apply(lambda q: q["id"][1])

q1_to_q2 = raw_df.copy().rename(columns={"q1": "text", "id1": "id", "id2": "dq_id"}).drop(columns=["questions", "q2"])
q2_to_q1 = raw_df.copy().rename(columns={"q2": "text", "id2": "id", "id1": "dq_id"}).drop(columns=["questions", "q1"])
flat_df = pd.concat([q1_to_q2, q2_to_q1])

flat_df.head()

,is_duplicate,text,id,dq_id
0,False,What is the step by step guide to invest in sh...,1,2
1,False,What is the story of Kohinoor (Koh-i-Noor) Dia...,3,4
2,False,How can I increase the speed of my internet co...,5,6
3,False,Why am I mentally very lonely? How can I solve...,7,8
4,False,"Which one dissolve in water quikly sugar, salt...",9,10


Next, we take a sample of the data and retaining just one row per question while storing a list of duplicated question IDs for each question.

In [6]:
df = flat_df.drop_duplicates("id").head(2000)
df["duplicated_questions"] = df["id"].apply(lambda qid: flat_df[flat_df["id"] == qid]["dq_id"].tolist())
df = df.drop(columns=["dq_id", "is_duplicate"])

df.tail()

,text,id,duplicated_questions
2003,What name will you give to a mathematics educa...,3985,"[3986, 535562]"
2004,What universities does Federal-Mogul recruit n...,3987,"[3988, 23638]"
2005,Why do people go for mba after masters in engi...,3989,"[3990, 161829, 316113, 168810]"
2006,How does a facebook account get hacked?,3991,"[3992, 379602, 35699]"
2007,Why do black people get upset when non-black r...,3993,[3994]


### Fit BM25 with Spacy Tokenizer

Here We'll create and fit a BM25 model using a simple split by space tokenization.

Since currently all available Python libraries for BM25 are also indexing the documents, we will use a simple and efficient hash-based encoding implementation from the `pinecone-text` library.

In [7]:
from pinecone_text.sparse import BM25Encoder

bm25 = BM25Encoder()

We need to calculate how often tokens appear in documents for BM25 to be able to create sparse vectors. To do this we call `bm25.fit` across our full dataset.

In [8]:
bm25.fit(df['text'])

  0%|          | 0/2000 [00:00<?, ?it/s]


### Dense Model

We use the popular [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) model available on Hugging Face for dense vectors.

In [9]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"running on {device}")

model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device=device
)

running on cpu


### Compute Dense & Sparse Embeddings

Create BM25 sparse embeddings:

In [10]:
df['sparse_values'] = df['text'].apply(bm25.encode_documents)

And now encode dense vector embeddings:

In [11]:
batch_size = 128
dense_values = []
for i in tqdm(range(0, len(df), batch_size)):
  dense_values += model.encode(df.iloc[i:i + batch_size]["text"].tolist()).tolist()

df['values'] = dense_values

  0%|          | 0/16 [00:00<?, ?it/s]

We organize our dataframe to align to the [pinecone-datasets](https://pypi.org/project/pinecone-datasets/) format:

In [12]:
df_result = df.copy()
df_result["metadata"] = None
df_result["blob"] = df_result.apply(lambda r: {"text": r["text"], "duplicated_questions": r["duplicated_questions"]}, axis=1)
df_result = df_result.drop(columns="text")

In [13]:
df_result.head()

,id,duplicated_questions,sparse_values,values,metadata,blob
0,1,[2],"{'indices': [2386266869, 3085233890, 550510908...","[0.06814990937709808, -0.03966414928436279, -0...",None,{'text': 'What is the step by step guide to in...
1,3,"[4, 282170, 380197, 488853]","{'indices': [1358668261, 1991459541, 930853874...","[-0.04679807275533676, 0.15511494874954224, -0...",None,{'text': 'What is the story of Kohinoor (Koh-i...
2,5,[6],"{'indices': [3570117707, 997512866, 3217550068...","[-0.02832488901913166, 0.037209585309028625, -...",None,{'text': 'How can I increase the speed of my i...
3,7,[8],"{'indices': [3705065656, 97715094, 930102859],...","[0.06325336545705795, -0.05639313533902168, 0....",None,{'text': 'Why am I mentally very lonely? How c...
4,9,"[10, 109465, 480204]","{'indices': [2257684172, 1522609561, 918197339...","[-0.04876847192645073, -0.025538966059684753, ...",None,{'text': 'Which one dissolve in water quikly s...


And now we have all we need to start using Pinecone vector database 🚀

For more details on that, check out [this notebook](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/bm25/bm25-quora.ipynb).